# Weighted Average

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 16].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [2]:
dataNoCvd = data[~(data[:,16] > 0.0)]
dataCvd = data[~(data[:,16] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [3]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [4]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 16, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 16), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, 16, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 16), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [5]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9


In [6]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [46]:
from xgboost import XGBClassifier


def my_fit(ratio,dataNoCvd,dataCvd,params):   
    Subarray=[]
    models = []
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0
    for i in range(numOfSubsamples):
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=1, gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=0, reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=1.0, verbosity=1)
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
#         print(len(Subarray))
#         print(len(Subarray[i]))
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 16, axis=1)
        y=np.delete(Subarray[i], slice(0, 16), axis=1)
        y=np.reshape(y, len(y))
        
        classifier.fit( X, y)
        models.append(classifier)
    return models   

In [47]:


def predict_weighted_average(models,X,weights):
    y_pred = []
    y_pred_models = []
#     weights = [0.16, 0.175, 0.2, 0.147, 0.148, 0.16]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
            
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred        
  

In [48]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params, weights):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 3, train_total_NoCvd[i], train_total_Cvd[i],params)


        y_pred = predict_weighted_average(models,X_test,weights)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [34]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        
params = {
                    "learning_rate"    :[ 0.1, 0.2, 0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[1,2, 3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100, 200, 300],
                     "min_child_weight"       :[0.5,0.75, 1.0],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0, 0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5,0.75, 1.0], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[1,2,3]
        }

In [19]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        
params = {
                    "learning_rate"    :[ 0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[300],
                     "subsample"       :[1.0],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3]
        }

In [23]:
comb = compute_comb(params)
print(comb[0])
weights = [0.11, 0.32, 0.12, 0.1, 0.25, 0.1]
accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,comb[0],weights)

[0.2, 2, 300, 1.0, 0, 0.5, 1, 3]


In [24]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.6678571428571428
0.6687028657616892
0.66
0.6643514328808446


In [95]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.11, 0.32, 0.12, 0.1, 0.25, 0.1]

0.6678571428571428
0.6687028657616892
0.66
0.6643514328808446


In [47]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.11, 0.32, 0.15, 0.1, 0.22, 0.1]

0.6678571428571428
0.6686651583710407
0.66
0.6643325791855204


In [43]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.1, 0.35, 0.15, 0.1, 0.2, 0.1]

0.6660714285714285
0.6686651583710407
0.64
0.6543325791855203


In [ ]:
parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
accuracy : 0.6678571428571428
spes     : 0.6725867269984918
sens     : 0.615
auc      : 0.6437933634992459
weights  : [0.1, 0.25, 0.2, 0.1, 0.2, 0.15]

In [ ]:
parameters : [0.3, 4, 100, 0.75, 0.5, 1.0, 2, 3]
accuracy : 0.6535714285714286
spes     : 0.6552036199095023
sens     : 0.64
auc      : 0.6476018099547511

In [149]:
# print(weight_scores[0])
k = weight_scores
sum_acc = 0
sum_sens = 0
sum_auc  =0
for i in range(6):
    acc = []
    sens = []
    auc = []
    for j in range(10):
        acc.append(weight_scores[i][j][0])
        sens.append(weight_scores[i][j][1])
        auc.append(weight_scores[i][j][2])
    acc = np.array(acc)
    sens= np.array(sens)
    auc = np.array(auc)
    print("accuracy :", acc.mean(), " sensitiv :", sens.mean(), " auc:", auc.mean())
    print(acc.mean()/3.5879545454545454,"   ", sens.mean()/2.3754007936507935, "    ", auc.mean()/3.482470238095238)
    print((acc.mean()*3.5879545454545454+ sens.mean()*2.3754007936507935+ auc.mean()*3.482470238095238)/9.445825577200576)
    print("--------------------------------------------------------------------------")
    sum_acc = sum_acc + acc.mean()
    sum_sens = sum_sens + sens.mean()
    sum_auc = sum_auc + auc.mean()
print("sum_acc ",sum_acc)
print("sum_sens",sum_sens)
print("sum_auc ",sum_auc)
print(sum_acc+sum_sens+sum_auc)

accuracy : 0.6067424242424242  sensitiv : 0.4366785714285714  auc: 0.6045386904761905
0.16910538206541245     0.18383363876772676      0.17359479023339686
0.5631629356276313
--------------------------------------------------------------------------
accuracy : 0.6399999999999999  sensitiv : 0.4618253968253968  auc: 0.6316964285714285
0.17837461202255017     0.19441998927499288      0.18139320234849712
0.5921319729584433
--------------------------------------------------------------------------
accuracy : 0.5639393939393939  sensitiv : 0.3386190476190476  auc: 0.5261309523809523
0.15717573530964293     0.14255238464352715      0.15107981300903334
0.4933375289673432
--------------------------------------------------------------------------
accuracy : 0.5574999999999999  sensitiv : 0.316531746031746  auc: 0.49117559523809523
0.1553810096915183     0.13325403733037533      0.1410422951688303
0.4724498431028997
--------------------------------------------------------------------------
accura

In [142]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.625
0.6339743589743589
0.515
0.5744871794871795


In [35]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bytree")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p] for i in list1  
                                    for j in list2 
                                    for k in list3 
                                    for l in list4 
                                    for m in list5 
                                    for n in list6 
                                    for o in list7
                                    for p in list8] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [36]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [37]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params,weights):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j],weights)
        if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print (style.BOLD + 'weights     ' + style.END, weights)
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        else:
            print(combs[j])
            print("accuracy    ",mean(accuracy))
            print("specificity ",mean(specificity))
            print("sensitivity ",mean(sensitivity))
            print("auc         ",mean(auc))
            print("weights     ",weights)
            
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc), weights])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

In [ ]:
#     weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
0.6589285714285714
0.6629713423831071
0.615
0.6389856711915536

# Ratio = 3

    parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
    accuracy : 0.6821428571428572
    spes     : 0.6879336349924585
    sens     : 0.615
    auc      : 0.6514668174962293


In [49]:
params = {
                    "learning_rate"    :[ 0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[1.0], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4]
        }

In [50]:
total= []
for k in range(5,45,5):
    kk = k/100
    for l in range(5,45,5):
        ll = l/100
        for m in range(5,45,5):
            mm = m/100
            for n in range(5,45,5):
                nn = n/100
                if kk+ll+mm+nn == 1:
                    weights = [kk, ll, mm, nn]
                    print(weights)
                    max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                    total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.05, 0.15, 0.4, 0.4]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.65
specificity  0.6648567119155354
sensitivity  0.47
auc          0.5674283559577677
weights      [0.05, 0.15, 0.4, 0.4]
[0.05, 0.2, 0.35, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6625
specificity  0.6745098039215687
sensitivity  0.52
auc          0.5972549019607842
weights      [0.05, 0.2, 0.35, 0.4]
----------------------------------------------------------------
[0.05, 0.2, 0.4, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.65
specificity  0.6610482654600301
sensitivity  0.52
auc          0.590524132730015
weights      [0.05, 0.2, 0.4, 0.35]
----------------------------------------------------------------
[0.05, 0.25, 0.3, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.66964285714

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6642857142857143
specificity  0.6744720965309201
sensitivity  0.54
auc          0.6072360482654601
weights      [0.1, 0.2, 0.35, 0.35]
----------------------------------------------------------------
[0.1, 0.2, 0.4, 0.3]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6517857142857143
specificity  0.6610105580693816
sensitivity  0.54
auc          0.6005052790346908
weights      [0.1, 0.2, 0.4, 0.3]
----------------------------------------------------------------
[0.1, 0.25, 0.25, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6821428571428572
specificity  0.6917797888386124
sensitivity  0.565
auc          0.6283898944193061
weights      [0.1, 0.25, 0.25, 0.4]
----------------------------------------------------------------
[0.1, 0.25

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6607142857142857
specificity  0.6705882352941177
sensitivity  0.54
auc          0.6052941176470589
weights      [0.1, 0.4, 0.4, 0.1]
----------------------------------------------------------------
[0.15, 0.05, 0.4, 0.4]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6482142857142857
specificity  0.6648567119155354
sensitivity  0.445
auc          0.5549283559577677
weights      [0.15, 0.05, 0.4, 0.4]
[0.15, 0.1, 0.35, 0.4]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6625
specificity  0.6783936651583711
sensitivity  0.47
auc          0.5741968325791855
weights      [0.15, 0.1, 0.35, 0.4]
[0.15, 0.1, 0.4, 0.35]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6428571428571429
specificity  0.6552036199095023
sensitivity  0.495
auc          0.5751018099547511
weights      [0.15, 0.1, 0.4, 0.35]
[0.15, 0.15, 0.3, 0.4]
1
0
------------------------------------------

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6785714285714286
specificity  0.6860482654600302
sensitivity  0.59
auc          0.638024132730015
weights      [0.15, 0.35, 0.3, 0.2]
----------------------------------------------------------------
[0.15, 0.35, 0.35, 0.15]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6642857142857143
specificity  0.6725113122171946
sensitivity  0.565
auc          0.6187556561085973
weights      [0.15, 0.35, 0.35, 0.15]
----------------------------------------------------------------
[0.15, 0.35, 0.4, 0.1]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6553571428571429
specificity  0.6628959276018099
sensitivity  0.565
auc          0.613947963800905
weights      [0.15, 0.35, 0.4, 0.1]
----------------------------------------------------------------
[0.

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6767857142857143
specificity  0.6860482654600302
sensitivity  0.565
auc          0.6255241327300151
weights      [0.2, 0.25, 0.15, 0.4]
----------------------------------------------------------------
[0.2, 0.25, 0.2, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6785714285714286
specificity  0.6860482654600302
sensitivity  0.59
auc          0.6380241327300151
weights      [0.2, 0.25, 0.2, 0.35]
----------------------------------------------------------------
[0.2, 0.25, 0.25, 0.3]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6732142857142858
specificity  0.6802790346907994
sensitivity  0.59
auc          0.6351395173453998
weights      [0.2, 0.25, 0.25, 0.3]
----------------------------------------------------------------
[0.2, 

...

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6392857142857143
specificity  0.6475490196078432
sensitivity  0.54
auc          0.5937745098039215
weights      [0.3, 0.2, 0.4, 0.1]
----------------------------------------------------------------
[0.3, 0.25, 0.05, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6732142857142857
specificity  0.6822021116138763
sensitivity  0.565
auc          0.6236010558069381
weights      [0.3, 0.25, 0.05, 0.4]
----------------------------------------------------------------
[0.3, 0.25, 0.1, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6785714285714286
specificity  0.6841251885369533
sensitivity  0.615
auc          0.6495625942684766
weights      [0.3, 0.25, 0.1, 0.35]
----------------------------------------------------------------
[0.3, 0

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6696428571428572
specificity  0.6763574660633485
sensitivity  0.59
auc          0.6331787330316742
weights      [0.3, 0.4, 0.25, 0.05]
----------------------------------------------------------------
[0.35, 0.05, 0.2, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6696428571428571
specificity  0.6802790346907994
sensitivity  0.54
auc          0.6101395173453997
weights      [0.35, 0.05, 0.2, 0.4]
----------------------------------------------------------------
[0.35, 0.05, 0.3, 0.3]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.65
specificity  0.664894419306184
sensitivity  0.47
auc          0.567447209653092
weights      [0.35, 0.05, 0.3, 0.3]
[0.35, 0.05, 0.35, 0.25]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6392857142857143
specificity  0.6552790346907994
sensitivity  0.445
auc     

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6482142857142857
specificity  0.6552413273001508
sensitivity  0.565
auc          0.6101206636500754
weights      [0.35, 0.2, 0.35, 0.1]
----------------------------------------------------------------
[0.35, 0.2, 0.4, 0.05]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6410714285714285
specificity  0.6514328808446456
sensitivity  0.515
auc          0.5832164404223228
weights      [0.35, 0.2, 0.4, 0.05]
----------------------------------------------------------------
[0.35, 0.25, 0.05, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6607142857142857
specificity  0.664894419306184
sensitivity  0.615
auc          0.639947209653092
weights      [0.35, 0.25, 0.05, 0.35]
----------------------------------------------------------------
[0

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.65
specificity  0.6610105580693816
sensitivity  0.515
auc          0.5880052790346908
weights      [0.4, 0.05, 0.25, 0.3]
----------------------------------------------------------------
[0.4, 0.05, 0.3, 0.25]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6482142857142857
specificity  0.6591251885369532
sensitivity  0.515
auc          0.5870625942684766
weights      [0.4, 0.05, 0.3, 0.25]
----------------------------------------------------------------
[0.4, 0.05, 0.35, 0.2]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6428571428571429
specificity  0.6533559577677225
sensitivity  0.515
auc          0.5841779788838612
weights      [0.4, 0.05, 0.35, 0.2]
----------------------------------------------------------------
[0.4, 0.05, 0.4, 0

----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6553571428571429
specificity  0.6610105580693816
sensitivity  0.59
auc          0.6255052790346908
weights      [0.4, 0.2, 0.15, 0.25]
----------------------------------------------------------------
[0.4, 0.2, 0.2, 0.2]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6607142857142857
specificity  0.6687028657616892
sensitivity  0.565
auc          0.6168514328808447
weights      [0.4, 0.2, 0.2, 0.2]
----------------------------------------------------------------
[0.4, 0.2, 0.25, 0.15]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.6535714285714286
specificity  0.6610105580693816
sensitivity  0.565
auc          0.6130052790346908
weights      [0.4, 0.2, 0.25, 0.15]
----------------------------------------------------------------
[0.4, 0.2

In [54]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.66:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy : 0.7
spes     : 0.7072398190045249
sens     : 0.615
auc      : 0.6611199095022624
weights  : [0.15, 0.4, 0.05, 0.4]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy : 0.6785714285714286
spes     : 0.6822021116138763
sens     : 0.64
auc      : 0.6611010558069381
weights  : [0.2, 0.4, 0.25, 0.15]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy : 0.6857142857142857
spes     : 0.6899321266968326
sens     : 0.64
auc      : 0.6649660633484162
weights  : [0.25, 0.4, 0.1, 0.25]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy : 0.6839285714285714
spes     : 0.6880090497737557
sens     : 0.64
auc      : 0.6640045248868778
weights  : [0.25, 0.4, 0.15, 0.2]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy : 0.678571428571428

## Καλύτερες τιμές:

    parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
    accuracy : 0.6892857142857143
    spes     : 0.6937782805429864
    sens     : 0.64
    auc      : 0.6668891402714932
    weights  : [0.35, 0.35, 0.1, 0.2]

    parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
    accuracy : 0.7
    spes     : 0.7072398190045249
    sens     : 0.615
    auc      : 0.6611199095022624
    weights  : [0.15, 0.4, 0.05, 0.4]

    parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
    accuracy : 0.6125
    spes     : 0.6110105580693816
    sens     : 0.64
    auc      : 0.6255052790346908


In [55]:
params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[1],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100],
                    "min_child_weight" :[5],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[1.0], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[5]
        }

In [56]:
total= []
for k in range(5,45,5):
    kk = k/100
    for l in range(5,45,5):
        ll = l/100
        for m in range(5,45,5):
            mm = m/100
            for n in range(5,45,5):
                nn = n/100
                if kk+ll+mm+nn == 1:
                    weights = [kk, ll, mm, nn]
                    print(weights)
                    max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                    total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.05, 0.15, 0.4, 0.4]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5910714285714286
specificity  0.5898190045248869
sensitivity  0.615
auc          0.6024095022624434
weights      [0.05, 0.15, 0.4, 0.4]
[0.05, 0.2, 0.35, 0.4]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5946428571428571
specificity  0.5936651583710407
sensitivity  0.615
auc          0.6043325791855203
weights      [0.05, 0.2, 0.35, 0.4]
[0.05, 0.2, 0.4, 0.35]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5892857142857143
specificity  0.58789592760181
sensitivity  0.615
auc          0.6014479638009049
weights      [0.05, 0.2, 0.4, 0.35]
[0.05, 0.25, 0.3, 0.4]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6013574660633484
sensitivity  0.59
auc          0.5956787330316742
weights      [0.05, 0.25, 0.3, 0.4]
[0.05, 0.25, 0.4, 0.3]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5964285714285714
specificity  0.5955882352941176
sensitivity  0.615
auc          0.6052941176470588
weights  

[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6032805429864253
sensitivity  0.565
auc          0.5841402714932127
weights      [0.1, 0.3, 0.25, 0.35]
[0.1, 0.3, 0.3, 0.3]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6089285714285714
specificity  0.6090497737556562
sensitivity  0.615
auc          0.6120248868778281
weights      [0.1, 0.3, 0.3, 0.3]
----------------------------------------------------------------
[0.1, 0.3, 0.35, 0.25]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6053571428571428
specificity  0.6052036199095022
sensitivity  0.615
auc          0.6101018099547512
weights      [0.1, 0.3, 0.35, 0.25]
----------------------------------------------------------------
[0.1, 0.3, 0.4, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6032805429864253
sensitivity  0.565
auc          0.5841402714932127
wei

----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6035714285714285
specificity  0.6032805429864253
sensitivity  0.615
auc          0.6091402714932127
weights      [0.15, 0.25, 0.35, 0.25]
----------------------------------------------------------------
[0.15, 0.25, 0.4, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5964285714285714
specificity  0.6013574660633484
sensitivity  0.54
auc          0.5706787330316742
weights      [0.15, 0.25, 0.4, 0.2]
[0.15, 0.3, 0.15, 0.4]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5964285714285714
specificity  0.601395173453997
sensitivity  0.54
auc          0.5706975867269984
weights      [0.15, 0.3, 0.15, 0.4]
[0.15, 0.3, 0.25, 0.3]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6013574660633484
sensitivity  0.59
auc          0.5956787330316742
weights      [0.15, 0.3, 0.25, 0.3]
[0.15, 0.3, 0.3, 0.25]
1
0
---------------------------------------------------

[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5982142857142857
specificity  0.6013574660633484
sensitivity  0.565
auc          0.5831787330316742
weights      [0.2, 0.15, 0.4, 0.25]
[0.2, 0.2, 0.2, 0.4]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6033182503770739
sensitivity  0.565
auc          0.584159125188537
weights      [0.2, 0.2, 0.2, 0.4]
[0.2, 0.2, 0.25, 0.35]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6107142857142858
specificity  0.6129336349924586
sensitivity  0.59
auc          0.6014668174962292
weights      [0.2, 0.2, 0.25, 0.35]
----------------------------------------------------------------
[0.2, 0.2, 0.3, 0.3]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6089285714285714
specificity  0.6109728506787331
sensitivity  0.59
auc          0.6004864253393665
weights      [0.2, 0.2, 0.3, 0.3]
----------------------

----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6017857142857143
specificity  0.6033182503770739
sensitivity  0.59
auc          0.5966591251885369
weights      [0.2, 0.4, 0.15, 0.25]
----------------------------------------------------------------
[0.2, 0.4, 0.2, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5982142857142857
specificity  0.5994343891402715
sensitivity  0.59
auc          0.5947171945701357
weights      [0.2, 0.4, 0.2, 0.2]
[0.2, 0.4, 0.25, 0.15]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6013574660633484
sensitivity  0.59
auc          0.5956787330316742
weights      [0.2, 0.4, 0.25, 0.15]
[0.2, 0.4, 0.35, 0.05]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6035714285714285
specificity  0.6071266968325791
sensitivity  0.565
auc          0.5860633484162896
weights      [0.2, 0.4, 0.35, 0.05]
------------------

[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6013574660633484
sensitivity  0.59
auc          0.5956787330316742
weights      [0.25, 0.25, 0.4, 0.1]
[0.25, 0.3, 0.05, 0.4]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6107142857142858
specificity  0.6110859728506788
sensitivity  0.615
auc          0.6130429864253394
weights      [0.25, 0.3, 0.05, 0.4]
----------------------------------------------------------------
[0.25, 0.3, 0.1, 0.35]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6035714285714285
specificity  0.6052790346907995
sensitivity  0.59
auc          0.5976395173453997
weights      [0.25, 0.3, 0.1, 0.35]
----------------------------------------------------------------
[0.25, 0.3, 0.15, 0.3]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6053571428571428
sp

[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5910714285714286
specificity  0.5917797888386124
sensitivity  0.59
auc          0.5908898944193062
weights      [0.3, 0.1, 0.35, 0.25]
[0.3, 0.1, 0.4, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5910714285714286
specificity  0.5937028657616893
sensitivity  0.565
auc          0.5793514328808447
weights      [0.3, 0.1, 0.4, 0.2]
[0.3, 0.15, 0.15, 0.4]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6142857142857143
specificity  0.6148567119155355
sensitivity  0.615
auc          0.6149283559577677
weights      [0.3, 0.15, 0.15, 0.4]
----------------------------------------------------------------
[0.3, 0.15, 0.25, 0.3]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6107142857142858
specificity  0.6090874811463046
sensitivity  0.64
auc          0.6245437405731523
weights      [0.3, 0.15, 0.25, 0.3]
--

----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6089285714285714
specificity  0.6110859728506788
sensitivity  0.59
auc          0.6005429864253393
weights      [0.3, 0.35, 0.05, 0.3]
----------------------------------------------------------------
[0.3, 0.35, 0.15, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6033182503770739
sensitivity  0.565
auc          0.584159125188537
weights      [0.3, 0.35, 0.15, 0.2]
[0.3, 0.35, 0.3, 0.05]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5928571428571429
specificity  0.5955882352941176
sensitivity  0.565
auc          0.5802941176470588
weights      [0.3, 0.35, 0.3, 0.05]
[0.3, 0.4, 0.05, 0.25]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6
specificity  0.6014705882352941
sensitivity  0.59
auc          0.595735294117647
weights      [0.3, 0.4, 0.05, 0.25]
[0.3, 0.4, 0.1, 0.2]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5982142857142857
specifi

----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6017857142857143
specificity  0.6033182503770739
sensitivity  0.59
auc          0.5966591251885369
weights      [0.35, 0.2, 0.4, 0.05]
----------------------------------------------------------------
[0.35, 0.25, 0.05, 0.35]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6178571428571429
specificity  0.6206259426847662
sensitivity  0.59
auc          0.605312971342383
weights      [0.35, 0.25, 0.05, 0.35]
----------------------------------------------------------------
[0.35, 0.25, 0.1, 0.3]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6107142857142858
specificity  0.6110105580693816
sensitivity  0.615
auc          0.6130052790346908
weights      [0.35, 0.25, 0.1, 0.3]
----------------------------------------------------------------
[0.35, 0.

----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6160714285714286
specificity  0.6148567119155355
sensitivity  0.64
auc          0.6274283559577677
weights      [0.4, 0.15, 0.05, 0.4]
----------------------------------------------------------------
[0.4, 0.15, 0.1, 0.35]
1
0
----------------------------------------------------------------
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.6017857142857143
specificity  0.601395173453997
sensitivity  0.615
auc          0.6081975867269984
weights      [0.4, 0.15, 0.1, 0.35]
----------------------------------------------------------------
[0.4, 0.15, 0.15, 0.3]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5964285714285714
specificity  0.5956259426847662
sensitivity  0.615
auc          0.6053129713423832
weights      [0.4, 0.15, 0.15, 0.3]
[0.4, 0.15, 0.2, 0.25]
1
0
[0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy     0.5875
specificity  0.5879336349924585
sensitivity  0.59
auc          0

In [57]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.63:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy : 0.6053571428571428
spes     : 0.601395173453997
sens     : 0.665
auc      : 0.6331975867269984
weights  : [0.35, 0.05, 0.2, 0.4]
------------------------------------------------
parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy : 0.6107142857142858
spes     : 0.6071644042232277
sens     : 0.665
auc      : 0.6360822021116139
weights  : [0.35, 0.1, 0.15, 0.4]
------------------------------------------------
parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy : 0.6107142857142857
spes     : 0.6071644042232278
sens     : 0.665
auc      : 0.6360822021116139
weights  : [0.4, 0.05, 0.15, 0.4]
------------------------------------------------
parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
accuracy : 0.625
spes     : 0.6244720965309201
sens     : 0.64
auc      : 0.63223604826546
weights  : [0.4, 0.2, 0.05, 0.35]
------------------------------------------------
4


## Καλύτερες τιμές:

    parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
    accuracy : 0.6107142857142858
    spes     : 0.6071644042232277
    sens     : 0.665
    auc      : 0.6360822021116139
    weights  : [0.35, 0.1, 0.15, 0.4]

    parameters : [0.5, 1, 100, 5, 0, 1.0, 1, 5]
    accuracy : 0.6107142857142857
    spes     : 0.6071644042232278
    sens     : 0.665
    auc      : 0.6360822021116139
    weights  : [0.4, 0.05, 0.15, 0.4]

    parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 4]
    accuracy : 0.6839285714285714
    spes     : 0.6917420814479638
    sens     : 0.59
    auc      : 0.6408710407239819


In [58]:
params = {
                    "learning_rate"    :[0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[1.0], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[5]
        }

In [59]:
total= []
for k in range(5,45,5):
    kk = k/100
    for l in range(5,45,5):
        ll = l/100
        for m in range(5,45,5):
            mm = m/100
            for n in range(5,45,5):
                nn = n/100
                if kk+ll+mm+nn == 1:
                    weights = [kk, ll, mm, nn]
                    print(weights)
                    max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                    total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.05, 0.15, 0.4, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6232142857142857
specificity  0.6321266968325792
sensitivity  0.515
auc          0.5735633484162896
weights      [0.05, 0.15, 0.4, 0.4]
----------------------------------------------------------------
[0.05, 0.2, 0.35, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6303571428571428
specificity  0.635972850678733
sensitivity  0.565
auc          0.6004864253393665
weights      [0.05, 0.2, 0.35, 0.4]
----------------------------------------------------------------
[0.05, 0.2, 0.4, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6285714285714286
specificity  0.6340497737556561
sensitivity  0.565
auc          0.599524886877828
weights      [0.05, 0.2, 0.4, 0.35]
--------------------------------------------------

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6321428571428571
specificity  0.6378959276018099
sensitivity  0.565
auc          0.6014479638009049
weights      [0.1, 0.2, 0.3, 0.4]
----------------------------------------------------------------
[0.1, 0.2, 0.35, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6392857142857143
specificity  0.6456259426847663
sensitivity  0.565
auc          0.605312971342383
weights      [0.1, 0.2, 0.35, 0.35]
----------------------------------------------------------------
[0.1, 0.2, 0.4, 0.3]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6178571428571429
specificity  0.622473604826546
sensitivity  0.565
auc          0.593736802413273
weights      [0.1, 0.2, 0.4, 0.3]
----------------------------------------------------------------
[0.1, 0.25, 0.25, 0

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6160714285714286
specificity  0.6205505279034691
sensitivity  0.565
auc          0.5927752639517345
weights      [0.1, 0.4, 0.35, 0.15]
----------------------------------------------------------------
[0.1, 0.4, 0.4, 0.1]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6089285714285714
specificity  0.6128582202111614
sensitivity  0.565
auc          0.5889291101055807
weights      [0.1, 0.4, 0.4, 0.1]
----------------------------------------------------------------
[0.15, 0.05, 0.4, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6160714285714286
specificity  0.6243966817496229
sensitivity  0.515
auc          0.5696983408748114
weights      [0.15, 0.05, 0.4, 0.4]
----------------------------------------------------------------
[0.15, 0.1, 0.

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6446428571428572
specificity  0.647473604826546
sensitivity  0.615
auc          0.631236802413273
weights      [0.15, 0.35, 0.15, 0.35]
----------------------------------------------------------------
[0.15, 0.35, 0.2, 0.3]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6392857142857143
specificity  0.6417043740573153
sensitivity  0.615
auc          0.6283521870286576
weights      [0.15, 0.35, 0.2, 0.3]
----------------------------------------------------------------
[0.15, 0.35, 0.25, 0.25]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6410714285714286
specificity  0.6436651583710408
sensitivity  0.615
auc          0.6293325791855203
weights      [0.15, 0.35, 0.25, 0.25]
----------------------------------------------------------------
[0.15,

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6214285714285714
specificity  0.6263574660633484
sensitivity  0.565
auc          0.5956787330316742
weights      [0.2, 0.15, 0.4, 0.25]
----------------------------------------------------------------
[0.2, 0.2, 0.2, 0.4]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6357142857142857
specificity  0.6378582202111613
sensitivity  0.615
auc          0.6264291101055807
weights      [0.2, 0.2, 0.2, 0.4]
----------------------------------------------------------------
[0.2, 0.2, 0.25, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6428571428571429
specificity  0.6455505279034691
sensitivity  0.615
auc          0.6302752639517345
weights      [0.2, 0.2, 0.25, 0.35]
----------------------------------------------------------------
[0.2, 0.2, 0.3

...

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6375
specificity  0.6398567119155354
sensitivity  0.615
auc          0.6274283559577677
weights      [0.3, 0.3, 0.1, 0.3]
----------------------------------------------------------------
[0.3, 0.3, 0.15, 0.25]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6375
specificity  0.6398567119155354
sensitivity  0.615
auc          0.6274283559577677
weights      [0.3, 0.3, 0.15, 0.25]
----------------------------------------------------------------
[0.3, 0.3, 0.2, 0.2]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6392857142857143
specificity  0.6417797888386124
sensitivity  0.615
auc          0.6283898944193061
weights      [0.3, 0.3, 0.2, 0.2]
----------------------------------------------------------------
[0.3, 0.3, 0.25, 0.15]
1
0
-------------

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6410714285714285
specificity  0.6457013574660634
sensitivity  0.59
auc          0.6178506787330317
weights      [0.35, 0.15, 0.1, 0.4]
----------------------------------------------------------------
[0.35, 0.15, 0.15, 0.35]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6321428571428571
specificity  0.6360105580693816
sensitivity  0.59
auc          0.6130052790346908
weights      [0.35, 0.15, 0.15, 0.35]
----------------------------------------------------------------
[0.35, 0.15, 0.2, 0.3]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6339285714285714
specificity  0.6360105580693816
sensitivity  0.615
auc          0.6255052790346908
weights      [0.35, 0.15, 0.2, 0.3]
----------------------------------------------------------------
[0.35, 0

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6357142857142857
specificity  0.637971342383107
sensitivity  0.615
auc          0.6264856711915535
weights      [0.35, 0.3, 0.05, 0.3]
----------------------------------------------------------------
[0.35, 0.3, 0.15, 0.2]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6392857142857142
specificity  0.6417420814479637
sensitivity  0.615
auc          0.6283710407239819
weights      [0.35, 0.3, 0.15, 0.2]
----------------------------------------------------------------
[0.35, 0.3, 0.3, 0.05]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6357142857142857
specificity  0.6379336349924585
sensitivity  0.615
auc          0.6264668174962292
weights      [0.35, 0.3, 0.3, 0.05]
----------------------------------------------------------------
[0.35, 0.35

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6142857142857143
specificity  0.6167797888386124
sensitivity  0.59
auc          0.6033898944193062
weights      [0.4, 0.1, 0.3, 0.2]
----------------------------------------------------------------
[0.4, 0.1, 0.35, 0.15]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6142857142857143
specificity  0.6167797888386124
sensitivity  0.59
auc          0.6033898944193061
weights      [0.4, 0.1, 0.35, 0.15]
----------------------------------------------------------------
[0.4, 0.1, 0.4, 0.1]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6125
specificity  0.6167797888386124
sensitivity  0.565
auc          0.5908898944193062
weights      [0.4, 0.1, 0.4, 0.1]
----------------------------------------------------------------
[0.4, 0.15, 0.05, 0.4]
1
0
---

----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.625
specificity  0.6283182503770739
sensitivity  0.59
auc          0.609159125188537
weights      [0.4, 0.25, 0.25, 0.1]
----------------------------------------------------------------
[0.4, 0.25, 0.3, 0.05]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6267857142857143
specificity  0.6302413273001508
sensitivity  0.59
auc          0.6101206636500754
weights      [0.4, 0.25, 0.3, 0.05]
----------------------------------------------------------------
[0.4, 0.3, 0.05, 0.25]
1
0
----------------------------------------------------------------
[0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy     0.6392857142857142
specificity  0.641817496229261
sensitivity  0.615
auc          0.6284087481146304
weights      [0.4, 0.3, 0.05, 0.25]
----------------------------------------------------------------
[0.4, 0.3, 0.1, 0.2]
1
0
--

In [61]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.643:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy : 0.6464285714285715
spes     : 0.6475490196078432
sens     : 0.64
auc      : 0.6437745098039216
weights  : [0.35, 0.4, 0.05, 0.2]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy : 0.6464285714285715
spes     : 0.6475490196078432
sens     : 0.64
auc      : 0.6437745098039216
weights  : [0.35, 0.4, 0.1, 0.15]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy : 0.6464285714285715
spes     : 0.6475490196078432
sens     : 0.64
auc      : 0.6437745098039216
weights  : [0.35, 0.4, 0.15, 0.1]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy : 0.6464285714285715
spes     : 0.6475113122171946
sens     : 0.64
auc      : 0.6437556561085973
weights  : [0.4, 0.4, 0.1, 0.1]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
accuracy : 0.6464285714285

## Καλύτερη τιμή:

    parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 5]
    accuracy : 0.6464285714285715
    spes     : 0.6475490196078432
    sens     : 0.64
    auc      : 0.6437745098039216
    weights  : [0.35, 0.4, 0.05, 0.2]

    parameters : [0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
    accuracy : 0.6571428571428571
    spes     : 0.660972850678733
    sens     : 0.615
    auc      : 0.6379864253393664


In [62]:
params = {
                    "learning_rate"    :[0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[1],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[300],
                    "min_child_weight" :[2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[1.0], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4]
        }

In [63]:
total= []
for k in range(5,45,5):
    kk = k/100
    for l in range(5,45,5):
        ll = l/100
        for m in range(5,45,5):
            mm = m/100
            for n in range(5,45,5):
                nn = n/100
                if kk+ll+mm+nn == 1:
                    weights = [kk, ll, mm, nn]
                    print(weights)
                    max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                    total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.05, 0.15, 0.4, 0.4]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6178571428571429
specificity  0.6244343891402715
sensitivity  0.54
auc          0.5822171945701358
weights      [0.05, 0.15, 0.4, 0.4]
----------------------------------------------------------------
[0.05, 0.2, 0.35, 0.4]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.625
specificity  0.6302036199095022
sensitivity  0.565
auc          0.5976018099547511
weights      [0.05, 0.2, 0.35, 0.4]
----------------------------------------------------------------
[0.05, 0.2, 0.4, 0.35]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6214285714285714
specificity  0.6263574660633484
sensitivity  0.565
auc          0.5956787330316742
weights      [0.05, 0.2, 0.4, 0.35]
--------------------------------------------------------

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6303571428571428
specificity  0.635972850678733
sensitivity  0.565
auc          0.6004864253393665
weights      [0.1, 0.2, 0.3, 0.4]
----------------------------------------------------------------
[0.1, 0.2, 0.35, 0.35]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6321428571428571
specificity  0.635972850678733
sensitivity  0.59
auc          0.6129864253393665
weights      [0.1, 0.2, 0.35, 0.35]
----------------------------------------------------------------
[0.1, 0.2, 0.4, 0.3]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6375
specificity  0.6417420814479637
sensitivity  0.59
auc          0.6158710407239819
weights      [0.1, 0.2, 0.4, 0.3]
----------------------------------------------------------------
[0.1, 0.25, 0.25, 0.4]
1
0

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6535714285714286
specificity  0.6590497737556561
sensitivity  0.59
auc          0.624524886877828
weights      [0.1, 0.4, 0.35, 0.15]
----------------------------------------------------------------
[0.1, 0.4, 0.4, 0.1]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6482142857142857
specificity  0.6571266968325792
sensitivity  0.54
auc          0.5985633484162896
weights      [0.1, 0.4, 0.4, 0.1]
----------------------------------------------------------------
[0.15, 0.05, 0.4, 0.4]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6267857142857143
specificity  0.6321266968325792
sensitivity  0.565
auc          0.5985633484162896
weights      [0.15, 0.05, 0.4, 0.4]
----------------------------------------------------------------
[0.15, 0.1,

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.65
specificity  0.6571266968325792
sensitivity  0.565
auc          0.6110633484162896
weights      [0.15, 0.35, 0.15, 0.35]
----------------------------------------------------------------
[0.15, 0.35, 0.2, 0.3]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6535714285714286
specificity  0.660972850678733
sensitivity  0.565
auc          0.6129864253393665
weights      [0.15, 0.35, 0.2, 0.3]
----------------------------------------------------------------
[0.15, 0.35, 0.25, 0.25]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6535714285714286
specificity  0.6571266968325792
sensitivity  0.615
auc          0.6360633484162896
weights      [0.15, 0.35, 0.25, 0.25]
----------------------------------------------------------------
[0.15, 0.35, 

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6392857142857142
specificity  0.6417420814479639
sensitivity  0.615
auc          0.6283710407239819
weights      [0.2, 0.15, 0.4, 0.25]
----------------------------------------------------------------
[0.2, 0.2, 0.2, 0.4]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6464285714285715
specificity  0.6532805429864253
sensitivity  0.565
auc          0.6091402714932127
weights      [0.2, 0.2, 0.2, 0.4]
----------------------------------------------------------------
[0.2, 0.2, 0.25, 0.35]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6446428571428571
specificity  0.6494343891402715
sensitivity  0.59
auc          0.6197171945701357
weights      [0.2, 0.2, 0.25, 0.35]
----------------------------------------------------------------
[0.2, 0.2

...

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6607142857142857
specificity  0.6686651583710407
sensitivity  0.565
auc          0.6168325791855204
weights      [0.3, 0.3, 0.1, 0.3]
----------------------------------------------------------------
[0.3, 0.3, 0.15, 0.25]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6571428571428571
specificity  0.660972850678733
sensitivity  0.615
auc          0.6379864253393666
weights      [0.3, 0.3, 0.15, 0.25]
----------------------------------------------------------------
[0.3, 0.3, 0.2, 0.2]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6517857142857143
specificity  0.6552036199095023
sensitivity  0.615
auc          0.6351018099547512
weights      [0.3, 0.3, 0.2, 0.2]
----------------------------------------------------------------
[0.3, 0.3, 

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6571428571428571
specificity  0.664894419306184
sensitivity  0.565
auc          0.614947209653092
weights      [0.35, 0.15, 0.1, 0.4]
----------------------------------------------------------------
[0.35, 0.15, 0.15, 0.35]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6553571428571429
specificity  0.6610105580693816
sensitivity  0.59
auc          0.6255052790346908
weights      [0.35, 0.15, 0.15, 0.35]
----------------------------------------------------------------
[0.35, 0.15, 0.2, 0.3]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6571428571428571
specificity  0.6629336349924585
sensitivity  0.59
auc          0.6264668174962292
weights      [0.35, 0.15, 0.2, 0.3]
----------------------------------------------------------------
[0.3

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6607142857142857
specificity  0.6686651583710408
sensitivity  0.565
auc          0.6168325791855204
weights      [0.35, 0.3, 0.05, 0.3]
----------------------------------------------------------------
[0.35, 0.3, 0.15, 0.2]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6482142857142857
specificity  0.6532805429864253
sensitivity  0.59
auc          0.6216402714932127
weights      [0.35, 0.3, 0.15, 0.2]
----------------------------------------------------------------
[0.35, 0.3, 0.3, 0.05]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6517857142857143
specificity  0.6590497737556561
sensitivity  0.565
auc          0.6120248868778281
weights      [0.35, 0.3, 0.3, 0.05]
----------------------------------------------------------------
[0.35

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6357142857142857
specificity  0.6417043740573152
sensitivity  0.565
auc          0.6033521870286576
weights      [0.4, 0.1, 0.3, 0.2]
----------------------------------------------------------------
[0.4, 0.1, 0.35, 0.15]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6321428571428571
specificity  0.6397812971342383
sensitivity  0.54
auc          0.5898906485671191
weights      [0.4, 0.1, 0.35, 0.15]
----------------------------------------------------------------
[0.4, 0.1, 0.4, 0.1]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6357142857142857
specificity  0.6436274509803922
sensitivity  0.54
auc          0.5918137254901961
weights      [0.4, 0.1, 0.4, 0.1]
----------------------------------------------------------------
[0.4, 0.15, 

----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6464285714285714
specificity  0.6532805429864253
sensitivity  0.565
auc          0.6091402714932127
weights      [0.4, 0.25, 0.25, 0.1]
----------------------------------------------------------------
[0.4, 0.25, 0.3, 0.05]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6446428571428572
specificity  0.6513197586726999
sensitivity  0.565
auc          0.6081598793363499
weights      [0.4, 0.25, 0.3, 0.05]
----------------------------------------------------------------
[0.4, 0.3, 0.05, 0.25]
1
0
----------------------------------------------------------------
[0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy     0.6642857142857143
specificity  0.6705882352941176
sensitivity  0.59
auc          0.6302941176470589
weights      [0.4, 0.3, 0.05, 0.25]
----------------------------------------------------------------
[0.4,

In [64]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.643:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
accuracy : 0.6678571428571428
spes     : 0.6706259426847663
sens     : 0.64
auc      : 0.6553129713423831
weights  : [0.4, 0.25, 0.1, 0.25]
------------------------------------------------
1


## Καλύτερη τιμή:

    parameters : [0.2, 1, 300, 2, 0, 1.0, 1.5, 4]
    accuracy : 0.6678571428571428
    spes     : 0.6706259426847663
    sens     : 0.64
    auc      : 0.6553129713423831
    weights  : [0.4, 0.25, 0.1, 0.25]